In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import math
from sqlalchemy import create_engine
import datetime
import time
import psycopg2
from psycopg2 import Error

In [2]:
try:
    # connecting to DB 
    connection=psycopg2.connect(user="shithi", password="tallykhata03012021_1234", host="192.168.168.52", port="5432", database="tallykhata")
    cursor=connection.cursor()
    
    # generating k-fold training data
    for i in range(5):
    
        # extracting data
        postgres_query='''
        -- building training/test dataset
        select 
            -- identifier
            mobile_no, 
            -- features
            days_with_tk, 
            week_2_active_days, week_3_active_days, 
            week_2_trt, week_3_trt, 
            week_2_tacs, week_3_tacs, 
            week_2_roam_days, week_3_roam_days, 
            week_2_spent_sec, week_3_spent_sec, 
            -- labels
            churn_cat
        from
            (select *, row_number() over(partition by churn_cat order by random()) seq
            from data_vajapora.help_e
            ) tbl1
        where 
            (churn_cat=1 and seq<=20000)
            or 
            (churn_cat=0 and seq<=60000)
        order by random(); 
        '''
        cursor.execute(postgres_query)

        # writing the data to a csv
        colnames=[desc[0] for desc in cursor.description]
        df_fetched=pd.DataFrame(cursor.fetchall(), columns=colnames)
        df_fetched.to_csv("daily_churn_pred_3_"+str(i+1)+".csv", index=False)
        
        print("Dataset generated: "+str(i+1))
    
# handling exception
except (Exception, psycopg2.Error) as error:
    print("Error!", error)
    
# closing connection to DB
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("\nPostgreSQL connection is safely closed.")

Dataset generated: 1
Dataset generated: 2
Dataset generated: 3
Dataset generated: 4
Dataset generated: 5

PostgreSQL connection is safely closed.
